<a href="https://colab.research.google.com/github/Sugoto/GPT-Model-with-RLHF/blob/main/GPT_Model_with_RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPT Architecture with RLHF on Shakespeare writing**

Let's train models for generating Shakespeare with different sentiments.

Happy Shakespeare will say things like: 

*“Nay, thanks, then, I do meet change, this Romeo. 
The pleasure of his hair!”*

Sad Shakespear will say things like:


*“The senators's dead, of their world:
Be not for your friends.”*



In [ ]:
!pip install tiktoken
!pip install transformers

In [2]:
import copy
import time
import tiktoken
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.distributions import Categorical

from transformers import pipeline

### Set Hyperparameters

In [3]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 3000 # maximum number of iterations for training
eval_interval = 100 # evaluation interval during training
learning_rate = 1e-3 # learning rate for the optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu' # device to use for computation (GPU if available, else CPU)
eval_iters = 200 # number of iterations to use for evaluation
n_embd = 64 # size of the embedding layer
n_head = 4 # number of heads in the multi-head attention layer
n_layer = 4 # number of layers in the transformer network
dropout = 0.0 # dropout probability

torch.manual_seed(1337)

### Download Tiny Shakespeare dataset

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2023-04-03 04:14:29--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.005s  

2023-04-03 04:14:29 (209 MB/s) - ‘input.txt’ saved [1115394/1115394]



### Using tiktoken - OpenAI's implementation of a Byte Pair Encoding (BPE) tokenizer

In [5]:
enc = tiktoken.get_encoding("gpt2")
vocab_size = 50257

print(enc.encode('sugoto'))
print(enc.encode('thh'))
print(enc.decode([31373]))

[82, 1018, 2069]
[400, 71]
hello


### Data Preparation and Loss Estimation

In [11]:
# Train and test splits
data = torch.tensor(enc.encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data, val_data = torch.utils.data.random_split(data, [n, len(data) - n])

# data loading
def get_batch(split):
    """
    Generates a small batch of data of inputs x and targets y.

    Args:
        split (str): Either 'train' or 'val' to indicate which dataset to use.

    Returns:
        Tuple[Tensor, Tensor]: The input tensor of shape (batch_size, block_size),
            and the target tensor of shape (batch_size, block_size).
    """
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    batch = torch.stack([data[i:i+block_size] for i in ix])
    x, y = batch[:, :-1], batch[:, 1:]
    return x.to(device), y.to(device)

@torch.no_grad()

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [9]:
class GPT(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [12]:
model = GPT()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

6.684497 M parameters
step 0: train loss 10.9975, val loss 10.9945
step 100: train loss 6.5189, val loss 6.5481
step 200: train loss 6.4546, val loss 6.4754
step 300: train loss 6.4198, val loss 6.4449
step 400: train loss 6.4077, val loss 6.4476
step 500: train loss 6.3808, val loss 6.4273
step 600: train loss 6.3731, val loss 6.4371
step 700: train loss 6.3846, val loss 6.4216
step 800: train loss 6.3649, val loss 6.4191
step 900: train loss 6.3652, val loss 6.4034
step 1000: train loss 6.3496, val loss 6.4137
step 1100: train loss 6.3540, val loss 6.4068
step 1200: train loss 6.3541, val loss 6.3836
step 1300: train loss 6.3469, val loss 6.4004
step 1400: train loss 6.3583, val loss 6.4321
step 1500: train loss 6.3385, val loss 6.4043
step 1600: train loss 6.3379, val loss 6.4045
step 1700: train loss 6.3431, val loss 6.4082
step 1800: train loss 6.3531, val loss 6.3986
step 1900: train loss 6.3314, val loss 6.3989
step 2000: train loss 6.3434, val loss 6.4172
step 2100: train loss 

In [13]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(enc.decode(m.generate(context, max_new_tokens=100)[0].tolist()))

! hundred any cons,
's
 have we be for so hisixt
 thatpt'
d,

bleAn thy is lord under winner!, messenger,Did hadUS than:CompareI Duke EL for vir Rome daughter visitNot chance then beand who see tongue: respect will- of and carries meet shall:, man hands
's to inn she'd:;'ll
 inj thoughtsWas: beOneALri with somewhatProv access with call me thrive


SayL


In [14]:
sentiment_pipeline = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [15]:
inps = ["I'm the king of the world!", 
        "I'll be back.",
        "The cake is a lie", 
        "To be forgotten is worse than death",
        "All happy families are alike; each unhappy family is unhappy in its own way.",
        "You don't need a reason to help people",
        ]
res = sentiment_pipeline(inps)

for i in range(len(inps)):
  res[i]['text'] = inps[i]
  print(res[i])

{'label': 'POS', 'score': 0.9771729707717896, 'text': "I'm the king of the world!"}
{'label': 'POS', 'score': 0.5481613874435425, 'text': "I'll be back."}
{'label': 'NEG', 'score': 0.7581188678741455, 'text': 'The cake is a lie'}
{'label': 'NEG', 'score': 0.820936381816864, 'text': 'To be forgotten is worse than death'}
{'label': 'NEU', 'score': 0.7874236702919006, 'text': 'All happy families are alike; each unhappy family is unhappy in its own way.'}
{'label': 'NEU', 'score': 0.8731083273887634, 'text': "You don't need a reason to help people"}


In [16]:
def get_reward(text, mode):
  sent = sentiment_pipeline(text)
  if mode == '+ve':
    labels = torch.tensor([a['label']=='POS' for a in sent],dtype=torch.float16).unsqueeze(-1).to(device)
  elif mode == '-ve':
    labels = torch.tensor([a['label']=='NEG' for a in sent],dtype=torch.float16).unsqueeze(-1).to(device)
  else:
    raise ValueError('Unknown Mode')
  
  weights = torch.tensor([a['score'] for a in sent],dtype=torch.float32).unsqueeze(-1).to(device)
  
  rewards = labels * weights # (B, 1)

  return rewards

In [17]:
def flatten(l):
    return [item for sublist in l for item in sublist]
print('Rewards in +ve mode')
list(zip(inps, flatten(get_reward(inps, '+ve').tolist())))


Rewards in +ve mode


[("I'm the king of the world!", 0.9771729707717896),
 ("I'll be back.", 0.5481613874435425),
 ('The cake is a lie', 0.0),
 ('To be forgotten is worse than death', 0.0),
 ('All happy families are alike; each unhappy family is unhappy in its own way.',
  0.0),
 ("You don't need a reason to help people", 0.0)]

In [18]:
print('Rewards in -ve mode')
list(zip(inps, flatten(get_reward(inps, '-ve').tolist())))

Rewards in -ve mode


[("I'm the king of the world!", 0.0),
 ("I'll be back.", 0.0),
 ('The cake is a lie', 0.7581188678741455),
 ('To be forgotten is worse than death', 0.820936381816864),
 ('All happy families are alike; each unhappy family is unhappy in its own way.',
  0.0),
 ("You don't need a reason to help people", 0.0)]

In [19]:
eval_interval_rlhf = 20
max_iters_rlhf = 1000


In [20]:

class RLHF(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, idx, targets=None):
        return self.model(idx, targets)
     
    def generate(self, idx, max_new_tokens, block_size, ref_model=None):
        # idx is (B, T) array of indices in the current context
        log_probs = torch.tensor([]).to(device)
        log_probs_ref = torch.tensor([]).to(device)
        
        for i in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]

            # get the predictions
            logits, loss = self(idx_cond)

            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # logits define instance of Iategorical class
            m = Categorical(logits=logits)
            
            # sample from the distribution
            idx_next = m.sample()
            
            # get the log probability and append to running sequence
            log_probs_idx_next = m.log_prob(idx_next)    
            log_probs = torch.cat((log_probs, log_probs_idx_next.view(-1,1)), dim=1)
            
            if ref_model is not None:
              # get log probability of sample idx_next under the reference model
              logits_ref, _ = ref_model(idx_cond)
              logits_ref = logits_ref[:, -1, :] # becomes (B, C)
            
              m_ref = Categorical(logits=logits_ref)
              log_probs_ref_idx_next = m_ref.log_prob(idx_next)    
              log_probs_ref = torch.cat((log_probs_ref, log_probs_ref_idx_next.view(-1,1)), dim=1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next.view(-1,1)), dim=1) # (B, T+1)

        return idx, log_probs, log_probs_ref

In [21]:

ref_model = copy.deepcopy(model)

In [ ]:

RLHFmodel = RLHF(model)
RLHFmodel.to(device)

ref_model.to(device)

actor_optimizer = torch.optim.AdamW(RLHFmodel.parameters(), lr=1e-3)
X, Y = get_batch('train') # fetch the very first batch
X = torch.ones((X.shape[0], 1), dtype=torch.long).to(device) # for now there is no prompt
X = X*enc.encode('The')[0] # start with ''The'
t0  = time.time()
max_new_tokens = block_size
rews_all = []
actor_loss_all = []
mode = '+ve'
ref_coef = 0.2
e_coef = 0.1
for iter in range(max_iters_rlhf):

  states, log_probs, log_probs_ref = RLHFmodel.generate(
      X, max_new_tokens, block_size, ref_model=ref_model)

  states = states[:,-max_new_tokens:]
  log_probs = log_probs[:,-max_new_tokens:] # (B, max_new_tokens)
  if ref_model is not None:
    log_probs_ref = log_probs_ref[:,-max_new_tokens:] # (B, max_new_tokens)
  
  rewards = get_reward([enc.decode(s.tolist()) for s in states], mode)
  
  pg = (rewards+ref_coef*log_probs_ref-e_coef*log_probs)* log_probs.squeeze()
  
  # log(1) = 0
  # -log(1/N) = log(N)

  # when ref_coef=e_coef this is equivalent to penalising for KL divergence
  # pg = (rewards-ref_coef*(log_probs-log_probs_ref)* log_probs.squeeze() 
  
  actor_loss = -pg.sum()

  actor_optimizer.zero_grad(set_to_none=True)
  actor_loss.backward()
  actor_optimizer.step()

  rews_all.append(rewards.mean().detach().cpu().numpy())
  actor_loss_all.append(actor_loss.detach().cpu().numpy())

  if iter % eval_interval_rlhf == 0:
      t1 = time.time()
      print('\n')
      print(f'iter: {iter}, time: {t1-t0}')
      print(f'Actor loss: {np.mean(actor_loss_all[-eval_interval_rlhf:])}')
      print(f'rets: {np.mean(rews_all[-eval_interval_rlhf:])}')

      textRLHF = RLHFmodel.generate(X, 2*max_new_tokens, block_size, ref_model=None)[0]
      for i in range(1):
          text_i = textRLHF[i,:]
          print(enc.decode(text_i.tolist()))




iter: 0, time: 3.3210113048553467
Actor loss: -2148.58349609375
rets: 0.10012473165988922
The departing
 hadNo brother
 You when notTo baby. know
 himself bring,AN they
self and laughter wait stands- mistress bats,essBERIOouPE ' let INtol the in in heartAL aI man this br on theK wellant use. and. been of gone ofpers


iter: 20, time: 75.81459379196167
Actor loss: -2273.817138671875
rets: 0.047431208193302155
TheWere!, Ice
land Suffolk theNowed: pardonTheenchUL: joy hisTR 'I with, toself days made, heir,
 beH thy's'd those
 pleased death, the beforeista rust, satisfaction
:
W none loveENThe
OW: we H came. Fro


iter: 40, time: 147.9004647731781
Actor loss: -2303.54443359375
rets: 0.03873438388109207
TheDen: insulting whoIROMastical: the wedUM seeks harmful prosper a purpose the head anchor. Rome shall the close per'll have his councils, purs G this I Rut
isTo shame haveIO
 OFsufficient be and meet jewels are of
 the
tis Romeo:is king call R captain

.


iter: 60, time: 220.84135293960

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(enc.decode(m.generate(context, max_new_tokens=200)[0].tolist()))

1. We trained a GPT model to reproduce Shakespeare
2. We built a reward model by repurposing a Huggingface sentiment classifier
3. We fine tuned the GPT model using reinforcement learning. 
4. The model over-optimised the reward so we penalised it for moving too far from a reference model
5. We found it to be far too repetitive and so we added in an entropy bonus to encourage diverse outputs

---





